<a href="https://colab.research.google.com/github/PetarSam/BTCprice-widget/blob/master/AI_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Package imports

In [ ]:
import pandas as pd
import regex as re
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
from sklearn import svm
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from collections import defaultdict
from nltk.corpus import wordnet as wn
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Data load

In [ ]:
URL = "https://raw.githubusercontent.com/Iuxta-ai/AI-dataset/master/dataset.csv?token=AIDXVT2WZ7AL5TDMG4A6MA27QL3VQ"

In [ ]:
df = pd.read_csv(URL, error_bad_lines=False)

In [ ]:
df.head()

,date,text,orig_price,30min_price,60min_price,90min_price,120min_price,150min_price,180min_price
0,2020-09-03 15:58:53,Congratulations to our friends Account for the...,1.1642,1.1443,1.1375,1.1433,1.1350,1.1410,1.1290
1,2020-09-01 16:29:35,Less than a month later and we already have ad...,1.4310,1.3864,1.4029,1.4098,1.3946,1.3580,1.3168
2,2020-08-27 16:34:48,We are happy to announce we concluded the refu...,1.1002,1.1234,1.1179,1.1143,1.1259,1.1126,1.1292
3,2020-08-25 14:07:19,We've just upgraded the exchange: now doing mu...,1.0882,1.0724,1.0799,1.0909,1.0901,1.0547,1.0581
4,2020-08-21 13:25:58,Great to have our friends Account using Balanc...,0.9166,0.9083,0.9328,0.9365,0.9306,0.9488,0.9262


In [ ]:
df_safe = df

# Cleaning proper nouns

In [ ]:
names = ["NEO", "Litecoin", "Qtum", "Cardano", "Ripple", "EOS", "MIOTA", "Stellar", "Ontology", "Tron", "Etherium Classic", "Icon", "Nuls", "Vechain", "Chainlink", "Waves", "BitTorrent", "SoMee.Social", "Holo", "Zilliqa", "0x", "Fetch.ai", "Bassic Attention Coin", "Monero", "Zcash", "IOST", "Celer Network", "Dash", "Nano", "OMG Network", "THETA", "Enjin Coin", "Matic Network", "Cosmos", "Harmony", "Algorand", "Elrond", "Dusk Network", "Ankr", "WINk", "Contentos", "Pundi X", "Cocos-BCX", "Metal", "TomoChain", "Dent", "Mainframe", "Selfkey", "Dock", "Wanchain", "FunFaur", "Civic", "Chiliz", "Band Protocol", "Beam ", "Ren", "Ravencoin", "HyperCash", "Hedera Hashgraph", "Blockstack", "Kava.io", "ARPA Chain", "iExec RLC", "TROY", "FTX Token", "Origin Protocol", "WazirX", "Lisk", "LTO Networks", "Stratis", "Aion", "COTI", "Standard Tokenatization Protocol", "Waltonchain", "Steamr", "Zcoin", "Cartesi", "Hive", "Chromia", "Ardor", "Aave", "Measurable Data Token", "Kyber Network", "Augur", "Loopring", "pNetwork", "Compound", "Siacoin", "Horizen", "Syntetix Network Token", "DigiByte", "Swipe", "Maker", "Neblio", "Bread", "Steem", "Bluzelle", "Solana", "IRISnet", "THORChain", "FIO", "Travala", "Balancer", "IQeon", "Tezos", "NKN", "Paxos", "Fantom", "Perlin", "Crypto.com", "StormX", "Decred", "Storj"]
names = [x.upper() for x in names]
print(names)

['NEO', 'LITECOIN', 'QTUM', 'CARDANO', 'RIPPLE', 'EOS', 'MIOTA', 'STELLAR', 'ONTOLOGY', 'TRON', 'ETHERIUM CLASSIC', 'ICON', 'NULS', 'VECHAIN', 'CHAINLINK', 'WAVES', 'BITTORRENT', 'SOMEE.SOCIAL', 'HOLO', 'ZILLIQA', '0X', 'FETCH.AI', 'BASSIC ATTENTION COIN', 'MONERO', 'ZCASH', 'IOST', 'CELER NETWORK', 'DASH', 'NANO', 'OMG NETWORK', 'THETA', 'ENJIN COIN', 'MATIC NETWORK', 'COSMOS', 'HARMONY', 'ALGORAND', 'ELROND', 'DUSK NETWORK', 'ANKR', 'WINK', 'CONTENTOS', 'PUNDI X', 'COCOS-BCX', 'METAL', 'TOMOCHAIN', 'DENT', 'MAINFRAME', 'SELFKEY', 'DOCK', 'WANCHAIN', 'FUNFAUR', 'CIVIC', 'CHILIZ', 'BAND PROTOCOL', 'BEAM ', 'REN', 'RAVENCOIN', 'HYPERCASH', 'HEDERA HASHGRAPH', 'BLOCKSTACK', 'KAVA.IO', 'ARPA CHAIN', 'IEXEC RLC', 'TROY', 'FTX TOKEN', 'ORIGIN PROTOCOL', 'WAZIRX', 'LISK', 'LTO NETWORKS', 'STRATIS', 'AION', 'COTI', 'STANDARD TOKENATIZATION PROTOCOL', 'WALTONCHAIN', 'STEAMR', 'ZCOIN', 'CARTESI', 'HIVE', 'CHROMIA', 'ARDOR', 'AAVE', 'MEASURABLE DATA TOKEN', 'KYBER NETWORK', 'AUGUR', 'LOOPRING', 

In [ ]:
def clean_proper_nouns(text):
  strings = str(text).split(' ')
  new_string = []
  for string in strings:
    if string.upper() in names:
      new_string.append("COINAME")
    else:
      new_string.append(string)
  return " ".join(new_string)

In [ ]:
df.text = df.text.apply(clean_proper_nouns)

## Finding Y column

In [ ]:
df.head()

,date,text,orig_price,30min_price,60min_price,90min_price,120min_price,150min_price,180min_price
0,2020-09-03 15:58:53,Congratulations to our friends Account for the...,1.1642,1.1443,1.1375,1.1433,1.1350,1.1410,1.1290
1,2020-09-01 16:29:35,Less than a month later and we already have ad...,1.4310,1.3864,1.4029,1.4098,1.3946,1.3580,1.3168
2,2020-08-27 16:34:48,We are happy to announce we concluded the refu...,1.1002,1.1234,1.1179,1.1143,1.1259,1.1126,1.1292
3,2020-08-25 14:07:19,We've just upgraded the exchange: now doing mu...,1.0882,1.0724,1.0799,1.0909,1.0901,1.0547,1.0581
4,2020-08-21 13:25:58,Great to have our friends Account using COINAM...,0.9166,0.9083,0.9328,0.9365,0.9306,0.9488,0.9262


In [ ]:
df.drop(['date', '30min_price','60min_price','90min_price','120min_price','150min_price'], axis=1, inplace=True)

In [ ]:
df['180delta'] = ((df["180min_price"]/df['orig_price'])-1)*100

In [ ]:
def val_to_class(val):
    if float(val) > 2.0:
        return 1
    else:
        return 0

In [ ]:
df['result'] = df.apply(lambda x: val_to_class(x['180delta']),axis=1)

In [ ]:
df.drop(['orig_price','180min_price',	'180delta'], axis=1, inplace=True)

## Cleaning rows with little text

In [ ]:
df.describe

<bound method NDFrame.describe of                                                     text  result
0      Congratulations to our friends Account for the...       0
1      Less than a month later and we already have ad...       0
2      We are happy to announce we concluded the refu...       1
3      We've just upgraded the exchange: now doing mu...       0
4      Great to have our friends Account using COINAM...       0
...                                                  ...     ...
42892  The #cryptoback on Account is available in the...       1
42893   Earn up to Coin crypto back with BlockFi!  Ea...       0
42894  The #cryptoback on Account is available in the...       1
42895  Use COINAME to earn #crypto when you #shop! Ch...       0
42896  Use COINAME to earn #crypto when you #shop! Ch...       0

[42897 rows x 2 columns]>

In [ ]:
df = df[df['text'].map(len) > 30]

In [ ]:
df.describe

<bound method NDFrame.describe of                                                     text  result
0      Congratulations to our friends Account for the...       0
1      Less than a month later and we already have ad...       0
2      We are happy to announce we concluded the refu...       1
3      We've just upgraded the exchange: now doing mu...       0
4      Great to have our friends Account using COINAM...       0
...                                                  ...     ...
42892  The #cryptoback on Account is available in the...       1
42893   Earn up to Coin crypto back with BlockFi!  Ea...       0
42894  The #cryptoback on Account is available in the...       1
42895  Use COINAME to earn #crypto when you #shop! Ch...       0
42896  Use COINAME to earn #crypto when you #shop! Ch...       0

[39620 rows x 2 columns]>

In [ ]:
df['text'] = df['text'].str.lower()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


# Additional processing

In [ ]:
df['text'] = [word_tokenize(entry) for entry in df['text']]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df.head()

,text,result
0,"[congratulations, to, our, friends, account, f...",0
1,"[less, than, a, month, later, and, we, already...",0
2,"[we, are, happy, to, announce, we, concluded, ...",1
3,"[we, 've, just, upgraded, the, exchange, :, no...",0
4,"[great, to, have, our, friends, account, using...",0


In [ ]:
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
for index,entry in enumerate(df['text']):
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    df.loc[index,'text_final'] = str(Final_words)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/

In [ ]:
df.head()

,text,result
0,"['congratulation', 'friend', 'account', 'launc...",0.0
1,"['less', 'month', 'later', 'already', 'add', '...",0.0
2,"['happy', 'announce', 'conclude', 'refund', 't...",1.0
3,"['upgrade', 'exchange', 'smarter', 'routing']",0.0
4,"['great', 'friend', 'account', 'use', 'coiname...",0.0


In [ ]:
df.drop(['text'], axis=1, inplace=True)

In [ ]:
df.rename(columns={'text_final':'text'}, inplace=True)

In [ ]:
df = df[['text', 'result']]

In [ ]:
df['text'] = [str(i) for i in df['text']]

# NLP - Pipeline

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['result'], test_size=0.3, random_state=60)

<30011x1500 sparse matrix of type '<class 'numpy.float64'>'
	with 327803 stored elements in Compressed Sparse Row format>

###Vectorization and TDIFD

In [ ]:
Tfidf_vect = TfidfVectorizer(max_features=1500, min_df=5, max_df=0.7)
Tfidf_vect.fit(df['text'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=0.7, max_features=1500,
                min_df=5, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [ ]:
X_train = Tfidf_vect.transform(X_train)
X_test = Tfidf_vect.transform(X_test)

AttributeError: ignored

In [ ]:
print(Tfidf_vect.vocabulary_)

{'congratulation': 269, 'friend': 550, 'account': 8, 'launch': 745, 'excited': 476, 'honor': 639, 'core': 289, 'new': 883, 'token': 1338, 'model': 851, 'less': 756, 'month': 859, 'later': 744, 'already': 48, 'add': 22, 'coin': 233, 'billion': 134, 'trade': 1358, 'special': 1224, 'thanks': 1318, 'amaze': 53, 'community': 248, 'partner': 950, 'make': 797, 'possible': 994, 'happy': 610, 'announce': 66, 'today': 1336, 'patience': 956, 'upgrade': 1399, 'exchange': 474, 'great': 590, 'use': 1405, 'coiname': 234, 'welcome': 1456, 'proposal': 1027, 'vote': 1437, 'go': 581, 'active': 17, 'monday': 854, 'august': 101, 'release': 1085, 'address': 25, 'recently': 1065, 'discover': 380, 'issue': 711, 'could': 292, 'allow': 44, 'node': 894, 'hit': 628, 'service': 1169, 'attack': 95, 'people': 963, 'run': 1130, 'well': 1457, 'miner': 842, 'pair': 943, 'available': 104, 'mark': 806, 'anniversary': 65, 'blockchain': 142, 'bring': 163, 'next': 887, 'year': 1486, 'beyond': 132, 'global': 579, 'network': 

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42873 entries, 0 to 39579
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   text    42873 non-null  object 
 1   result  39620 non-null  float64
dtypes: float64(1), object(1)
memory usage: 2.2+ MB


### Classification algorithms

Naive Bayes

In [ ]:
X_train.info()

AttributeError: ignored

In [ ]:
# fit the training dataset on the NB classifier
Naive = naive_bayes.MultinomialNB()
Naive.fit(X_train, y_train)
# predict the labels on validation dataset
predictions_NB = Naive.predict(X_test)
# Use accuracy_score function to get the accuracy
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, y_test)*100)

ValueError: ignored

## Old Code

In [ ]:
text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])

NameError: ignored

In [ ]:
text_clf.fit(X_train, y_train)

NameError: ignored

In [ ]:
y_predicted = text_clf.predict(X_test)

In [ ]:
accuracy = mean_absolute_error(y_test, y_predicted)
print(1 - accuracy)

In [ ]:
clf = svm.SVC()

In [ ]:
text_clf2 = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', svm.SVC())])

In [ ]:
text_clf2.fit(X_train, y_train)

In [ ]:
y_predicted2 = text_clf2.predict(X_test)

In [ ]:
accuracy = mean_absolute_error(y_test, y_predicted2)
print(1 - accuracy)